# Analyzing the logs

In [1]:
# Run from the top of the repository
%cd ..

/raid/home/pbachant/calkit/boom-paper


In [2]:
import pandas as pd
import polars as pl
import yaml
import itertools
import os

with open("calkit.yaml") as f:
    calkit_config = yaml.safe_load(f)

# Load the BOOM and Kowalski configurations
boom_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-boom"]["iterate_over"]:
    boom_params[i["arg_name"]] = i["values"]

kowalski_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-kowalski"][
    "iterate_over"
]:
    kowalski_params[i["arg_name"]] = i["values"]


# Form the matrix of parameters for each
param_names = list(boom_params.keys())
param_vals = list(boom_params.values())
vals_product = list(itertools.product(*param_vals))
results_boom = []

for vals in vals_product:
    current_params = dict(zip(param_names, vals))
    boom_config = (
        f"na={current_params['n_alert_workers']}-"
        f"nml={current_params['n_ml_workers']}-"
        f"nf={current_params['n_filter_workers']}"
    )
    boom_consumer_log_fpath = f"logs/boom-{boom_config}/consumer.log"
    boom_scheduler_log_fpath = f"logs/boom-{boom_config}/scheduler.log"
    # To calculate BOOM wall time, take first timestamp from the consumer log
    # as the start and the last timestamp of the scheduler as the end
    if not os.path.isfile(boom_consumer_log_fpath):
        print(f"WARNING: {boom_consumer_log_fpath} does not exist")
        continue
    with open(boom_consumer_log_fpath) as f:
        line = f.readline()
        t1_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    with open(boom_scheduler_log_fpath) as f:
        lines = f.readlines()
        line = lines[-3]
        t2_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    current_params["start_time"] = t1_b
    current_params["end_time"] = t2_b
    boom_wall_time = t2_b - t1_b
    results_boom.append(current_params)

df_boom = pl.DataFrame(results_boom).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Kowalski only has one parameter name, so we don't need the product
param_name = list(kowalski_params.keys())[0]
param_vals = kowalski_params[param_name]
results_kowalski = []

for val in param_vals:
    res = {param_name: val}
    kowalski_config = f"n={val}"
    log_fpath = f"logs/kowalski-{kowalski_config}/supervisord.log"
    if not os.path.isfile(log_fpath):
        print(f"WARNING: {log_fpath} does not exist")
        continue
    # To calculate Kowalski wall time, just use the supervisord logs
    with open(log_fpath) as f:
        lines = f.readlines()
        for line in lines:
            if "alert-broker-ztf entered RUNNING state" in line:
                t1_k = pd.to_datetime("T".join(line.split()[:2]))
                break
        for line in lines:
            if "received SIGTERM indicating exit request" in line:
                t2_k = pd.to_datetime("T".join(line.split()[:2]))
                break
    # Print the total time of each, in minutes
    kowalski_wall_time = t2_k - t1_k  # type: ignore
    res["start_time"] = t1_k
    res["end_time"] = t2_k
    results_kowalski.append(res)

df_kowalski = pl.DataFrame(results_kowalski).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Write results to file for later analysis
os.makedirs("results", exist_ok=True)
df_boom.write_csv("results/boom.csv")
df_kowalski.write_csv("results/kowalski.csv")

# Calculate throughput factor from a specific config for each
boom_wall_time = df_boom.filter(
    (pl.col("n_alert_workers") == 3)
    & (pl.col("n_ml_workers") == 3)
    & (pl.col("n_filter_workers") == 1)
).select("wall_time_s").row(0)[0]
kowalski_wall_time = (
    df_kowalski.filter(pl.col("n_workers") == 7)
    .select(pl.col("wall_time_s"))
    .row(0)[0]
)

boom_throughput_factor = kowalski_wall_time / boom_wall_time
print(f"BOOM throughput factor: {boom_throughput_factor:.1f}")

BOOM throughput factor: 6.6


In [3]:
pl.Config.set_tbl_rows(50)
df_boom

n_alert_workers,n_ml_workers,n_filter_workers,start_time,end_time,wall_time_s
i64,i64,i64,"datetime[μs, UTC]","datetime[μs, UTC]",i64
1,3,1,2025-08-05 14:26:24.213941 UTC,2025-08-05 14:29:38.751073 UTC,194
1,3,3,2025-08-05 14:30:13.437025 UTC,2025-08-05 14:33:27.536604 UTC,194
1,6,1,2025-08-05 14:34:01.390505 UTC,2025-08-05 14:37:19.484783 UTC,198
1,6,3,2025-08-05 14:37:53.417456 UTC,2025-08-05 14:41:03.689921 UTC,190
1,9,1,2025-08-05 14:41:39.977386 UTC,2025-08-05 14:44:51.470072 UTC,191
1,9,3,2025-08-05 14:45:25.583645 UTC,2025-08-05 14:48:42.174141 UTC,196
3,3,1,2025-08-05 14:49:18.670749 UTC,2025-08-05 14:50:33.246925 UTC,74
3,3,3,2025-08-05 14:51:04.392720 UTC,2025-08-05 14:52:20.420558 UTC,76
3,6,1,2025-08-05 14:52:51.703040 UTC,2025-08-05 14:53:55.742715 UTC,64


In [4]:
df_kowalski

n_workers,start_time,end_time,wall_time_s
i64,datetime[μs],datetime[μs],i64
3,2025-08-05 15:08:04,2025-08-05 15:26:01,1077
6,2025-08-05 15:26:32,2025-08-05 15:35:47,555
7,2025-08-05 15:36:17,2025-08-05 15:44:26,489
9,2025-08-05 15:44:56,2025-08-05 15:52:42,466
12,2025-08-05 15:53:11,2025-08-05 16:00:55,464
15,2025-08-05 16:01:25,2025-08-05 16:09:09,464


In [5]:
# TODO: Put actual results here
kafka_ingest_strate_mbps = "XXXXX"
kafka_ingest_factor = "XXXXX"

template = r"""% This file was automatically generated; edits will be overwritten!
\newcommand{\boomthroughputfactor}{BOOM_THROUGHPUT_FACTOR}
\newcommand{\kakfaingestratembps}{KAFKA_INGEST_STRATE_MBPS}
\newcommand{\kakfaingestfactor}{KAFKA_INGEST_FACTOR}
"""

# This might be better with an f-string, but this is TeX and who wants to deal
# with escaping all the braces?
template = (
    template.replace(
        "BOOM_THROUGHPUT_FACTOR", f"{round(boom_throughput_factor):1d}"
    )
    .replace("KAFKA_INGEST_STRATE_MBPS", kafka_ingest_strate_mbps)
    .replace("KAFKA_INGEST_FACTOR", kafka_ingest_factor)
)

with open("paper/results.tex", "w") as f:
    f.write(template)